### 贝叶斯要解决的问题：逆向概率

P（A|B）=P(B|A)P(A)/P(B)

先验概率：P(A)

## 垃圾邮件过滤实例：

P（h+|D）=P（h+）P（D|h+）/P（D）   
P（h-|D）=P（h-）P（D|h-）/P（D）

P（D|h+）=P（d1,d2,...,dn|h+）=P（d1|h+）P（d2|d1,h+）P（d3|d2,d1,h+）...   

朴素贝叶斯假设特征之间独立、互不影响，所以：   
P（D|h+）=P（d1|h+）P（d2|h+）P（d3|h+）...

# 实例：贝叶斯实现拼写检查器

### 求解：argmaxc P（c|w）->argmaxc P（w|c）P（c）/P（w）###   

* P(c),文章中出现一个正确拼写词c的概率，也就是，在英文文章中，c出现的概率有多大  
* P(w|c),在用户想输入c的情况下，敲成w的概率，也就是，用户会有多大概率将c敲错成w  
* argmaxc，用来枚举所有可能的c并且选取概率最大的

In [19]:
import re,collections

#把语料库中的单词都抽取出来，转成小写，去除单词中的特殊符号
def words(text): return re.findall("[a-z]+",text.lower())

def train(features):
    #lambda:1代表词频很小的
    model=collections.defaultdict(lambda:1)
    for f in features:
        model[f]+=1
    return model

NWORDS=train(words(open("big.txt").read()))

### 编辑距离： ###    

两个词之间的编辑距离定义为使用了几次插入（在词中间插入一个单字母）、删除（删除一个单字母）、交换（交换相邻两个字母）、替换（字母替换）的操作，使得从一个词变成另一个词

In [49]:
#返回所有与单词w编辑距离为1的集合
def edits1(word):
    n=len(word)
    return set([word[0:i]+word[i+1:] for i in range(n)]+                            #删除一个字母
              [word[0:i]+word[i+1]+word[i]+word[i+2:]  for i in range(n-1)]+       #相邻两个字母交换
               [word[0:i]+c+word[i+1:]  for i in range(n) for c in alphabet]+       #替换一个字母
               [word[0:i]+c+word[i:]  for i in range(n+1) for c in alphabet]       #插入一个字母
              )

In [50]:
#与“something”距离为1的单词有494个
print(len(edits1("something")))

494


In [51]:
#返回所有与单词w编辑距离为2的集合
#在这些编辑距离小于2 的词中间，只把那些正确的词作为候选词
def edits2(word):
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1))

In [52]:
#与“something”距离为2的单词有114324个
print(len(edits2("something")))

114324


优化：只将正确的单词作为候选词（不然把那些错误单词也加入的话、候选词太多了）

In [53]:
def known_edits2(word):
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1) if e2 in NWORDS)

In [55]:
#优化后、与“something”距离为2的单词仅剩4个（含自身）
print(len(known_edits2("something")))

print(known_edits2("something"))

{'soothing', 'something', 'seething', 'smoothing'}


In [62]:
def known(words):
    return set(w for w in words if w in NWORDS)

def correct(word):
    candidates=known([word]) or known(edits1(word)) or known_edits2(word) or [word]
    return max(candidates,key=lambda w:NWORDS[w])

In [65]:
print(correct("somthing"))
print("——"*15)
print(correct("appli"))
print("——"*15)

something
——————————————————————————————
apply
——————————————————————————————
